# tf.Dataset, batch를 활용한 loader 만들기

In [1]:
import tensorflow as tf

daily_sales_numbers = [21, 22, -100, 31, -1, 32, 34, 31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [2]:
for sales in tf_dataset:
    print(sales)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-100, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


### as_numpy_iterator

In [3]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-100
31
-1
32
34
31


### take

In [4]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-100


### filter

In [5]:
tf_dataset = tf_dataset.filter(lambda x: x > 0)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


### map

- Dataset 전체에 함수를 맵핑합니다.

In [6]:
tf_dataset = tf_dataset.map(lambda x: x * 2)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

42
44
62
64
68
62


### shuffle
- 데이터세트는 buffer_size 요소로 버퍼를 채운 다음이 버퍼에서 요소를 무작위로 샘플링하여 선택한 요소를 새 요소로 바꿉니다.

- 완벽한 셔플 링을 위해서는 데이터 세트의 전체 크기보다 크거나 같은 버퍼 크기가 필요합니다.

- 예를 들어, 데이터 집합에 10,000 개의 element가 있지만 buffer_size가 1,000으로 설정된 경우 셔플은 처음에 버퍼의 처음 1,000 개 element 중 임의의 element 만 선택합니다.

- element가 선택되면 버퍼의 공간이 다음 element (즉, 1,001-st)로 대체되어 1,000 element 버퍼를 유지합니다.

In [7]:
tf_dataset = tf_dataset.shuffle(10)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

62
42
64
62
68
44


In [8]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[44 64]
[68 62]
[62 42]


In [9]:
for sales_batch in tf_dataset.batch(3):
    print(sales_batch.numpy())

[62 64 62]
[68 42 44]


In [10]:
for sales in tf_dataset.batch(4):
    print(sales.numpy())

[44 42 68 64]
[62 62]


In [11]:
for sales in tf_dataset.batch(4, drop_remainder=True):
    print(sales.numpy())

[68 62 62 64]


- 위 모든 것을 한 줄로 결합

In [12]:
daily_sales_numbers = [21, 22, -100, 31, -1, 32, 34, 31]
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x: x > 0).map(lambda y: y * 2).shuffle(3).batch(4, drop_remainder=True)

for sales_batch in tf_dataset.as_numpy_iterator():
    print(sales_batch)

[62 64 42 44]


### train, test dataset 생성
- train : test = 8 : 2

In [17]:
daily_sales_numbers = list(range(100))

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

ds_count = len(tf_dataset)
train_size = int(ds_count * 0.8)

train_ds = tf_dataset.take(train_size)
test_ds  = tf_dataset.skip(train_size)

len(train_ds), len(test_ds)

(80, 20)